In [1]:
import pandas as pd;
import numpy as np;
import re
import scipy as sp;
import sklearn;
import sys;
from nltk.corpus import stopwords;
import nltk;
from gensim.models import ldamodel
import gensim.corpora;
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;
import pickle;
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

nltk.download('wordnet')
%matplotlib inline

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/markespina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
df = pd.read_csv('/Users/markespina/Downloads/movie-dialogs-corpus/movies_dialogs_corpus.csv')

In [11]:
# drop nulls and redundant columns
df.drop('Unnamed: 0', 1, inplace=True)
df.dropna(inplace=True)
df.reset_index(inplace=True)

In [12]:
df.head()

,index,ids,title,year,genre,rating,dialogue
0,0,m0,10 things i hate about you,1999,"['comedy', 'romance']",6,BIANCA Did you change your hair? CHASTITY No...
1,1,m1,1492 conquest of paradise,1992,"['adventure', 'biography', 'drama', 'history']",6,"ISABEL Is that the man I knew, Treasurer Sanc..."
2,2,m2,15 minutes,2001,"['action', 'crime', 'drama', 'thriller']",6,EMIL Just do what I do. Say the same thing I...
3,3,m3,2001 a space odyssey,1968,"['adventure', 'mystery', 'sci-fi']",8,"FLOYD How do you do, Mr. Miller? MILLER I'm ..."
4,4,m4,48 hrs,1982,"['action', 'comedy', 'crime', 'drama', 'thrill...",6,"GANZ Maybe you shoulda stole a better truck, ..."


In [13]:
# remove Capital Character names from scripthigh character counts occurances interefere with topic modelling
for x in tqdm(range(df.shape[0])):
    text = df['dialogue'][x]
    pattern = r"[A-Z][A-Z]\w+"
    df['dialogue'][x] = re.sub(pattern, "", text)

  0%|          | 0/594 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
100%|██████████| 594/594 [00:38<00:00, 15.60it/s]


In [14]:
# lowercase all text
df['dialogue'] = df['dialogue'].apply(lambda x: x.lower())

In [15]:
df.head()

,index,ids,title,year,genre,rating,dialogue
0,0,m0,10 things i hate about you,1999,"['comedy', 'romance']",6,did you change your hair? no. you might ...
1,1,m1,1492 conquest of paradise,1992,"['adventure', 'biography', 'drama', 'history']",6,"is that the man i knew, treasurer sanchez? ..."
2,2,m2,15 minutes,2001,"['action', 'crime', 'drama', 'thriller']",6,just do what i do. say the same thing i say...
3,3,m3,2001 a space odyssey,1968,"['adventure', 'mystery', 'sci-fi']",8,"how do you do, mr. miller? i'm terribly so..."
4,4,m4,48 hrs,1982,"['action', 'comedy', 'crime', 'drama', 'thrill...",6,"maybe you shoulda stole a better truck, tont..."


In [16]:
df2 = pd.read_csv('/Users/markespina/Downloads/movie-dialogs-corpus/movie_dialogs_stopwords_removed.csv')

In [19]:
df['dialogue'] = df2['dialogue']

In [20]:
X_train, X_test, y_train, y_test =train_test_split(df[['dialogue']], df['genre'], test_size=.25, random_state=42)


In [323]:
new_df = data_text
train_headlines = [value[0] for value in new_df.iloc[0:].values]

In [324]:
num_topics = 10

In [23]:
df['dialogue']

0      did change hair? no. might wanna think missed ...
1      is man knew, treasurer sanchez? yes, majesty. ...
2      just do. say thing say. open mouth. okay. fool...
3      how do, mr. miller? terribly sorry. way meet y...
4      maybe shoulda stole better truck, tonto. got r...
5      lord forgive me.. already know too, much.. ".....
6      welcome home. know much missed you? smoking ? ...
7      something wrong stairs? avoid-all-contact-day....
8      hey, wrong let see smile. kind long night. dan...
9      i sorry screw-up. help it late go trick-or-tre...
10     we already inspected. sir, plane carries presi...
11     commissioner, know mercury shuttle needs anoth...
12     jack, fred bliffert blue turtleneck? maybe fli...
13     hope plumbing same. is. go? put wash-and-wear ...
14     - today? terrible. - terrible? better yesterda...
15     what want? father vogler. chaplain here. thoug...
16     are cold? yes. good. jack. david. good time yo...
17     you carrying much money 

In [50]:
X_train, X_test = train_test_split(df['dialogue'], test_size=0.4, random_state=0)

vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )


#Applying the vectorizer
dialog_tfidf=vectorizer.fit_transform(df['dialogue'])

In [59]:
X_train.index[0]

299

In [60]:
#Applying the vectorizer
#dialog_tfidf=vectorizer.fit_transform(X_train['dialogue'])
print("Number of features: %d" % dialog_tfidf.get_shape()[1])

#splitting into training and test sets
X_train_tfidf, X_test_tfidf= train_test_split(dialog_tfidf, test_size=0.4, random_state=0)


#Reshapes the vectorizer output into something people can read
X_train_tfidf_csr = X_train_tfidf.tocsr()

#number of paragraphs
n = X_train_tfidf_csr.shape[0]
#A list of dictionaries, one per movie
tfidf_bymovie = [{} for _ in range(0,n)]
#List of features
terms = vectorizer.get_feature_names()
#for each paragraph, lists the feature words and their tf-idf scores
for i, j in zip(*X_train_tfidf_csr.nonzero()):
    tfidf_bymovie[i][terms[j]] = X_train_tfidf_csr[i, j]

#Keep in mind that the log base 2 of 1 is 0, so a tf-idf score of 0 indicates that the word was present once in that sentence.
print('Original dialog:', X_train[X_train.index[0]])
print('Tf_idf vector:', pd.Series(tfidf_bymovie[0]).sort_values(ascending=False))


Number of features: 47182
Original dialog: i cramps. believe let talk this. come on, double dating since fourth grade. yeah even look happened: michael mortenson kissed billy sullivan threw worm me. well going happen tonight. george said carlton nice guy. translation: total geek. anything better mark. shrink says bad. shrink always gives bad advice. hears choose tell him. mark asshole, cheated, borrowed money never paid back, never regular job. talented musician. every woman point date musician. wish get rid mark good. every time break see going out. guess weakness him. big brown bambi eyes. look eyes. wonder like you. noticed time. people notice frida. asked bartender "coyote ugly" meant. like "bagger" system. know, two- bagger someone ugly need two bags one bag put head another one case blows off. three-bagger two bags them, one bag head case two fall off. said one word me. maybe shy. date always pays attention to me. frida, mean criticism, might want talk around men. sorry. bad late

In [57]:
pd.Series(tfidf_bymovie[0]).sort_values(ascending=False)

doolittle      0.495320
frida          0.482937
talby          0.269551
werewolf       0.239751
carlton        0.178621
bomb           0.161592
mark           0.126702
pinback        0.121298
detonation     0.107369
gregory        0.106022
detonate       0.104958
peter          0.095669
asteroids      0.086681
george         0.082450
commander      0.074350
powell         0.069235
bagger         0.067388
boxers         0.067388
date           0.066723
nebula         0.065387
jennifer       0.064755
phoenix        0.063792
veil           0.058310
hairy          0.056757
malfunction    0.054131
pillowcase     0.053910
benito         0.053910
key            0.053614
lloyd          0.052150
pee            0.051753
                 ...   
paper          0.003884
cover          0.003876
earth          0.003868
floor          0.003868
teach          0.003868
service        0.003868
fix            0.003860
gives          0.003860
usually        0.003852
stuck          0.003844
lying          0

In [62]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

#Our SVD data reducer.  We are going to reduce the feature space from 25540 to 150.
svd= TruncatedSVD(150)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data, then project the training data.
X_train_lsa = lsa.fit_transform(X_train_tfidf)

variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

#Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
movies_by_component=pd.DataFrame(X_train_lsa,index=X_train)
for i in range(5):
    print('Component {}:'.format(i))
    print(movies_by_component.loc[:,i].sort_values(ascending=False)[0:2])




Percent variance captured by all components: 63.78787785711758
Component 0:
dialogue
on hunger strike, huh? eat last? thing yesterdayor today. sending meals regularly? yessir. refuses all. jam throat? quite simple. dealt prisoners time, one absurd! fuss snip girl. going see myself. eating? want eat! one thing want! definitely! see you. know next move is? cigarettes. put chains? might. right! put chains! anything want! going eat thing let boat! come on, ellie. stop silly. know going way. stand it! stand running life! insist it! ought know why. because yes. know. daughter love me. want make mistakes. because marrying fool king westley is wasting time. already married him. far concerned, not. yes? smart, you! subtle. gandhi chef like paul, would change whole political situation india. tempt me. hear? eat! please. fight empty stomach. remember napoleon said. hope comparing napoleon. strategist. idea strategy use lead pipe. taking me? south america. south america! leave miami hour. soo

In [553]:
df_clean = df_clean.merge(df[['ids','year', 'rating']],on='ids')

In [ ]:
df_clean.head()

In [68]:
nlp = spacy.load('en')

In [75]:
patterns = [r"--","\'", "[\[].*?[\]]", "[\?\.\,\!]"]

In [64]:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    for pattern in patterns:
        text = re.sub(pattern,' ',text)
    
    text = ' '.join(text.split())
    return text

In [9]:
# apply function to block of text
def preprocessor(func, text):    
    return" ".join((map(func, text.split())))

In [10]:
from tqdm import tqdm

In [13]:
df = df.reset_index()

In [65]:
import re
df_clean = pd.DataFrame()
df_clean=df

In [67]:
df_clean['dialogue'] =df_clean['dialogue'].apply(lambda x: x.replace("...",  ' '))

In [86]:
my_stopwords = list(set(stopwords.words('english'))) +['know, like']

In [110]:
sorted(my_stopwords)

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'get',
 'got',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'know',
 'like',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'oursel

In [69]:
df_clean = df_clean[df_clean.char_count<1000000]

AttributeError: 'DataFrame' object has no attribute 'char_count'

In [70]:
df_clean = df_clean.reset_index()

In [87]:
for x in tqdm(range(df_clean.shape[0])):   
    for word in my_stopwords:
        df_clean['dialogue'][x] = df_clean['dialogue'][x].replace(" "+word+" ", ' ')

  0%|          | 0/583 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
100%|██████████| 583/583 [1:48:28<00:00, 10.72s/it]


In [88]:
df_clean.to_csv('/Users/markespina/Downloads/movie-dialogs-corpus/movie_dialogs_stopwords_removed.csv')

In [94]:
nlp =spacy.load('en')

In [95]:
processed_dialogue =[]

In [71]:
df_clean['docs'] = df_clean['dialogue'].apply(nlp)

In [81]:
from collections import Counter


In [71]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

In [72]:
lda = LDA()

In [123]:
df_clean['genre'] = df_clean['genre'].astype(str)

In [563]:
from ast import literal_eval
df_clean['genre']
df_clean['genre_lists'] = df_clean.genre.apply(literal_eval)

In [566]:
df_clean['num_of_genres'] = df_clean['genre_lists'].apply(len)

In [570]:
def to_decade(year):
    if year > 1999:
        return '2000s'
    elif (year > 1989) & (year < 2000):
        return '1990s'
    elif (year > 1979) & (year < 1990):
        return '1980s'
    elif (year > 1969) & (year < 1980):
        return '1970s'
    elif (year > 1959) & (year < 1970):
        return '1960s'
    elif (year > 1949) & (year < 1960):
        return '1950s'
    elif (year > 1939) & (year < 1950):
        return '1940s'
    elif (year > 1929) & (year < 1940):
        return '1930s'
    else:
        return '1920s'

In [572]:
df_clean['release_decade'] = df_clean['year'].apply(to_decade)

In [574]:
df_clean.release_decade.value_counts()

1990s    237
2000s    136
1980s    106
1970s     49
1960s     19
1950s     15
1930s     15
1940s     15
1920s      2
Name: release_decade, dtype: int64

In [580]:
df_clean = df_clean[df_clean.num_of_genres>0]
df_clean['primary_genre'] = df_clean['genre_lists'].apply(lambda x: x[0])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [585]:
top_genres = list(df_clean.primary_genre.value_counts().index[:5])

In [588]:
outliers = df_clean[df_clean.primary_genre.isin(top_genres)==False]
outliers.reset_index(inplace=True)

In [623]:
df_clean.reset_index(inplace=True)

In [624]:
niche_titles = []
for x in tqdm(range(df_clean.shape[0])):

    idx =0
    while df_clean['primary_genre'][x] not in top_genres:
        try:
            df_clean['primary_genre'][x] = df_clean['genre_lists'][x][idx]
            idx+=1
        except IndexError:
            print('film {} is too niche'.format(df_clean['title'][x]))
            break
            
for x in tqdm(range(df_clean.shape[0])):

    if df_clean['primary_genre'][x] not in top_genres:
        niche_titles.append(df_clean['title'][x])

  0%|          | 0/592 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
 18%|█▊        | 104/592 [00:00<00:03, 137.82it/s]

film jaws is too niche


 18%|█▊        | 109/592 [00:02<00:44, 10.80it/s] 

film knight moves is too niche


 19%|█▉        | 112/592 [00:04<02:26,  3.28it/s]

...

film twelve monkeys is too niche


 98%|█████████▊| 581/592 [02:19<00:04,  2.48it/s]

film the witching hour is too niche


100%|██████████| 592/592 [00:00<00:00, 33337.29it/s]

film the wizard of oz is too niche


In [625]:
niche_titles

['jaws',
 'knight moves',
 'legend',
 'my mother dreams the satan s disciples in new york',
 'napoleon',
 'panic room',
 'back to the future',
 'basic instinct',
 'crazy love',
 'dark city',
 'escape from the planet of the apes',
 'fantastic voyage',
 'five feet high and rising',
 'hider in the house',
 'jaws 2',
 'jaws 3 d',
 'jaws the revenge',
 'jennifer eight',
 'klute',
 'misery',
 'arcade',
 'the nightmare before christmas',
 'playback',
 'red white black blue',
 'sleepy hollow',
 'spare me',
 'star trek the motion picture',
 'the third man',
 'the magic toyshop',
 'twelve monkeys',
 'the witching hour',
 'the wizard of oz']

In [629]:
df_clean[df_clean.title.isin(niche_titles)==False].head()

,index,ids,title,genre,dialogue,docs,key_words,year,rating,genre_lists,num_of_genres,release_decade,primary_genre
0,0,m0,10 things i hate about you,"['comedy', 'romance']",did change hair? no. might wanna think missed ...,"(did, change, hair, ?, no, ., might, wanna, th...",me it you her what something sister daddy him ...,1999,6,"[comedy, romance]",2,1990s,comedy
1,1,m1,1492 conquest of paradise,"['adventure', 'biography', 'drama', 'history']","is man knew, treasurer sanchez? yes, majesty. ...","(is, man, knew, ,, treasurer, sanchez, ?, yes,...",me you them us it man colon something ocean go...,1992,6,"[adventure, biography, drama, history]",4,1990s,drama
2,2,m2,15 minutes,"['action', 'crime', 'drama', 'thriller']",just do. say thing say. open mouth. okay. fool...,"(just, do, ., say, thing, say, ., open, mouth,...",me you it her him us what something money way ...,2001,6,"[action, crime, drama, thriller]",4,2000s,action
3,4,m4,48 hrs,"['action', 'comedy', 'crime', 'drama', 'thrill...","maybe shoulda stole better truck, tonto. got r...","(maybe, shoulda, stole, better, truck, ,, tont...",it you money me us jack him luther guy em way ...,1982,6,"[action, comedy, crime, drama, thriller]",5,1980s,action
4,5,m5,the fifth element,"['action', 'adventure', 'romance', 'sci-fi', '...","lord forgive me.. already know too, much.. "".....","(lord, forgive, me, .., already, know, too, ,,...",it me you stones father life leeloo president ...,1997,7,"[action, adventure, romance, sci-fi, thriller]",5,1990s,action


In [672]:
data = df_clean[df_clean.title.isin(niche_titles)==False]


In [811]:
vectorizer = TfidfVectorizer(max_df=0.8, # drop words that occur in more than 75 percent of the paragraphs
                             min_df=5, # only use words that appear at least n times
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case 
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )


#Applying the vectorizer
lines_tfidf=vectorizer.fit_transform(data['dialogue'])

In [812]:
print("Number of features: %d" % lines_tfidf.get_shape()[1])


Number of features: 21151


In [813]:
terms = vectorizer.get_feature_names()

In [815]:
lines_csr = lines_tfidf.tocsr()

#number of paragraphs
n = lines_csr.shape[0]
#A list of dictionaries, one per paragraph
tfidf_bypara = [{} for _ in range(0,n)]

#for each paragraph, lists the feature words and their tf-idf scores
for i, j in zip(*lines_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = lines_csr[i, j]


In [ ]:
for i in range(5):
    print('Original sentence:', data.loc[i,'key_words'])
    print('Tf_idf vector:', tfidf_bypara[i])

In [816]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

#Our SVD data reducer.  We are going to reduce the feature space from 7738 to the number below.
svd= TruncatedSVD(1500)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data, then project the training data.
lines_lsa = lsa.fit_transform(lines_tfidf)

variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

#Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
paras_by_component=pd.DataFrame(lines_lsa)
#for i in range(5):
#    print('Component {}:'.format(i))
#    print(paras_by_component.loc[:50,i].sort_values(ascending=False)[0:5])

Percent variance captured by all components: 100.00000000000009


In [817]:

paras_by_component.head(10)

,0,1,2,3,4,5,6,7,8,9,...,550,551,552,553,554,555,556,557,558,559
0,0.239908,-0.003827,-0.023690,-0.024658,-0.026567,-0.027365,-0.020369,0.063061,-0.016294,0.050185,...,-0.000023,-0.000049,-0.000020,-3.813596e-05,-1.438299e-05,0.000018,8.157259e-07,0.000007,-7.506876e-06,-2.258950e-06
1,0.142566,-0.022093,0.081804,-0.004820,0.020934,-0.027735,0.048107,-0.024822,-0.011719,-0.015273,...,-0.000010,0.000016,-0.000042,5.643952e-06,-1.185299e-05,-0.000019,-3.304703e-07,-0.000010,3.810290e-06,7.993421e-06
2,0.347904,-0.004925,-0.091814,0.040556,-0.034824,-0.008816,-0.022744,0.007261,0.038416,-0.058242,...,0.000036,-0.000025,0.000047,-6.667803e-05,-2.174240e-05,-0.000027,3.628005e-05,-0.000015,1.641572e-06,6.792923e-06
3,0.235854,-0.011261,-0.108006,0.086541,-0.036145,0.048612,0.053726,0.020430,0.007103,-0.052813,...,0.000034,-0.000023,0.000038,2.384010e-05,-1.867706e-05,0.000023,-9.137130e-06,-0.000016,-4.777278e-06,1.380395e-06
4,0.265387,-0.020108,0.073029,-0.008924,0.027031,-0.034707,-0.015527,-0.027596,-0.028797,-0.060380,...,-0.000037,0.000011,0.000019,-1.290758e-05,-7.024711e-06,0.000028,4.120757e-06,-0.000007,-2.408645e-07,2.828484e-07
5,0.307615,-0.003597,-0.058921,-0.014306,-0.015784,-0.042767,-0.048346,-0.068515,0.033318,-0.025251,...,0.000012,0.000053,-0.000034,-8.450265e-07,-1.489433e-05,-0.000014,1.096711e-05,0.000013,-1.693476e-05,-5.378713e-06
6,0.140440,0.003789,-0.030343,-0.018213,0.004086,0.002186,-0.045608,0.065914,0.036142,0.023371,...,0.000096,0.000040,0.000046,-2.638752e-04,2.459346e-05,0.000020,-8.347827e-06,-0.000019,3.955648e-05,2.120264e-06
7,0.729856,-0.046088,0.094741,-0.109562,-0.012430,-0.054439,-0.009114,-0.051839,-0.073750,-0.087841,...,0.007340,0.011351,-0.013558,4.204030e-02,-2.117422e-05,-0.000040,-3.193960e-03,-0.000069,3.306261e-05,-6.598111e-06
8,0.311090,-0.024589,-0.022554,-0.045675,-0.011489,-0.062026,0.008056,-0.037146,-0.016903,0.010603,...,-0.000364,-0.000712,0.000235,-1.382066e-03,-4.024820e-05,-0.000653,-2.505095e-03,0.000269,2.690163e-05,1.535502e-04
9,0.239881,-0.034392,0.130016,0.029164,0.042373,-0.079448,0.043520,-0.030843,-0.062106,-0.093955,...,-0.000168,-0.000275,0.000225,-6.572275e-04,-1.919623e-07,-0.000254,-1.012091e-03,0.000138,6.636528e-06,5.312218e-05


In [845]:
data= data.merge(df[['ids', 'clean_year', 'clean_rating']], on='ids')

In [846]:
paras_by_component = paras_by_component.reset_index(drop=True)
data=data.reset_index().drop('index',1)
new_data = data[['clean_year', 'clean_rating']].join(paras_by_component)


In [924]:
y = data['primary_genre']
for i,num in y.value_counts().iteritems():
    print("{}: {:.2f}%".format(i, (num/len(y)*100)))

drama: 30.36%
action: 25.89%
comedy: 23.57%
crime: 12.14%
horror: 8.04%


In [920]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
 
words = data['dialogue'][0].split()
ps = PorterStemmer()
 
for word in words:
    print(ps.stem(word))

did
chang
hair?
no.
might
wanna
think
miss
you.
say
expos
group
freshmen
girls.
bratwurst.
eat
lunch.
teeth
zipper?
new
guy?
tell
c
mon.
suppos
give
tour.
dakota
from?
north,
actually.
?
kidding.
peopl
actual
live
there?
yeah.
couple.
outnumb
cows,
though.
mani
peopl
old
school?
thirty-two.
get
out!
mani
peopl
go
here?
coupl
thousand.
evil
use
to.
yeah,
guy
never
seen
horse.
jack
clint
eastwood.
girl
burn,
pine,
perish?
she?
bianca
stratford.
sophomore.
even
think
not?
could
start
haircut,
matter.
allow
date
older
sister
does.
impossibility.
katarina
stratford.
my,
my.
terror
ms.
blais
again.
express
opinion
terrorist
action.
well,
yes,
compar
choic
express
year,
today
event
quit
mild.
way,
bobbi
rictor
gonad
retriev
oper
went
quit
well,
case
interested.
still
maintain
kick
balls.
mere
spectator.
point
kat
peopl
perceiv
somewhat
tempestuous?
believ
"heinou
bitch"
term
use
often.
that?
patrick
verona
random
skid.
pat
verona?
one
gone
year?
heard
porn
movies.
sure
complet
incap
anyth
int

normal,
normal.
normal?
bogey
lowenstein
parti
normal,
busi
listen
bitch
need
prozac
know
that.
forget
one
night
complet
wretched?
least
clout
fen-
suck
hedge-pig.
like
suppos
know
even
means.
shakespeare.
mayb
heard
him?
yeah,
freak
friend
mandella
boyfriend.
guess
sinc
allow
go
out,
obsess
dead
guy,
too.
oh,
god.
starting.
party.
daddy.
wear
belli
go.
daddy,
no!
minut
complet
unbalanced.
go
now?
promis
talk
boy
unless
sister
present.
why?
scare
away.
ya
goin?
away.
sister
here?
leav
sister
alone.
would
that?
this?
"i
get
trashed,
man."
suppos
party?
say,
wanna
do.
funny,
one
go?
here.
who?
joey.
hi.
look
like
thing
work
tonight,
huh?
know
chastity?
believ
share
art
instructor
great
would
mind
get
drink,
cameron?
extrem
unfortun
maneuver.
hell
that?
kind
guy
pick
girl
carri
away
talk
her?
buttholu
extremus.
hey,
make
progress.
no,
not.
yeah,
got
sear
catalog
thing
go
tube
sock
gig
"
gonna
huge.
ad
queen
harri
next
week.
queen
harry?
gay
cruis
line,
be,
like,
wear
uniform
stuff.
neat
a

In [848]:
X_train, X_test, y_train, y_test = train_test_split(new_data, y, test_size=.25, random_state =42)

In [849]:
from sklearn.model_selection import cross_val_score

from sklearn import ensemble

In [850]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [851]:
rfc = RandomForestClassifier(max_depth=25,max_features=75,n_estimators=800)

In [972]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features=75, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [974]:
rfc.score(X_test, y_test)
cross_val_score(rfc, X_test, y_test)

array([0.3125    , 0.17021277, 0.37777778])

In [826]:
gb =GradientBoostingClassifier()

In [899]:
gb =GradientBoostingClassifier(max_depth =4,min_impurity_decrease=0.05 )

In [969]:
gb.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=4,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.05, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [971]:
gb.score(X_test, y_test)
cross_val_score(gb, X_test, y_test)

array([0.20833333, 0.23404255, 0.2       ])

In [957]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier

In [962]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [968]:
lr.score(X_test, y_test)
cross_val_score(lr, X_test, y_test, cv=5)

array([0.4       , 0.34482759, 0.21428571, 0.2962963 , 0.46153846])

In [964]:
rgc = RidgeClassifier()
rgc.fit(X_train, y_train)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001)

In [966]:
rgc.score(X_test, y_test)

0.35

In [79]:
def get_keywords(text):
    count_nouns = Counter()
    for noun in text.noun_chunks:
        noun = str(noun).strip()
        if noun not in count_nouns.keys():
            count_nouns[noun] = 1
        else:
            count_nouns[noun] +=1

    keywords = [word for word, count in count_nouns.most_common()]

    return " ".join(keywords)

In [82]:
df_clean['key_words'] = df_clean['docs'].apply(get_keywords)


In [162]:
comedy = df_clean[df_clean.genre.str.contains('comedy')]
#X = df_clean['dialogue']
X = comedy['dialogue']

In [163]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(X) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 8.59 s, sys: 356 ms, total: 8.94 s
Wall time: 9.06 s
(157, 1893)


In [203]:
terms = tfidf_vectorizer.get_feature_names()

In [87]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

In [93]:
my_stopwords = list(set(stopwords.words('english'))) +["know", "well", "get", "got", "yeah", "like"]

In [89]:
def profanity_filter(word):
    bad_words = ['fuck', 'shit','damn']
    check =[]
    for bad_word in bad_words:
        if word.startswith(bad_word)==True:
            return True
        else:
            return False
    

In [107]:
n_top_words = 1000
k = 3

lda = LDA(n_topics=k, random_state=100)

id_topic = lda.fit_transform(tfidf_matrix)

topic_words = {}
vocab = tfidf_vectorizer.get_feature_names()
for topic, comp in enumerate(lda.components_):
    # for the n-dimensional array "arr":
    # argsort() returns a ranked n-dimensional array of arr, call it "ranked_array"
    # which contains the indices that would sort arr in a descending fashion
    # for the ith element in ranked_array, ranked_array[i] represents the index of the
    # element in arr that should be at the ith index in ranked_array
    # ex. arr = [3,7,1,0,3,6]
    # np.argsort(arr) -> [3, 2, 0, 4, 5, 1]
    # word_idx contains the indices in "topic" of the top num_top_words most relevant
    # to a given topic ... it is sorted ascending to begin with and then reversed (desc. now)    
    word_idx = np.argsort(comp)[::-1][:n_top_words]
    #print(topic)
    # store the words most relevant to the topic
    topic_words[topic] = [vocab[i] for i in word_idx if vocab[i] not in my_stopwords and len(vocab[i])> 3 and profanity_filter(vocab[i])==False]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [109]:
id_topic[300]

array([0.01452068, 0.97095885, 0.01452046])

In [1462]:

topic_labels = []
for weights in id_topic:
    for i, weight in enumerate(weights):
        if weight == max(weights):
            topic_labels.append(i)
            break


In [1463]:
test = df_clean[['ids', 'title', 'key_words']]
test['topic'] = topic_labels

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [1464]:
test.topic.value_counts()

2    401
0    124
5     38
3     29
Name: topic, dtype: int64

In [100]:
topics_count = []
for topic, words in topic_words.items():
    words
    print('Topic: %d' % topic)
    print('  %s' % ', '.join(words))
    x = Counter(words)
    topics_count.append(x)

Topic: 0
  shit, ship, jesus, says, goddamn, lost, father, army, charlie, word, gold, james, story, actually, brown, brought, women, chief, west, killed, blow, island, guys, plane, george, john, power, speak, shut, worry, somebody, jack, land, damn, gotta, house, command, hour, half, school, point, problem, truth, round, bank, hello, dark, lead, saying, business, eyes, stuff, thirty, town, balls, send, police, girls, high, drink, suppose, wife, ride, boat, door, telling, paid, trouble, bought, know know, different, slow, alright, turn, short, happy, boys, year, baby, nature, couple, water, times, relax, bring, reason, deal, york, bullshit, happening, true, younger, hours, month, evil, tied, break, hate, afraid, goes, shot, king, easy, want know, carrying, stand, bitch, rest, read, beautiful, people like, president, man know, week, soon, doors, captain, change, guest, attitude, society, turns, dinner, crazy, leaving, know gonna, cause, piece, young, kids, comes, bucks, release, freak, s

In [137]:
from sklearn.cluster import KMeans

In [196]:
num_clusters = 3

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 6.53 s, sys: 35.1 ms, total: 6.57 s
Wall time: 6.63 s


In [197]:
#from sklearn.externals import joblib

In [198]:
joblib.dump(km,  '/Users/markespina/Desktop/imdb_clusters/doc_cluster.pkl')
km = joblib.load('/Users/markespina/Desktop/imdb_clusters/doc_cluster.pkl')
clusters = km.labels_.tolist()

In [199]:
#df_clean[df_clean.genre.str.contains('(comedy)')].shape[0]

In [200]:
#new_df = df_clean[['title', 'year', 'genre','key_words', 'rating']].set_index([clusters])
new_df = comedy[['title', 'year', 'genre','key_words', 'rating']].set_index([clusters])

In [201]:
new_df['clusters'] = clusters

In [202]:
new_df['clusters'].value_counts()

0    76
2    43
1    38
Name: clusters, dtype: int64

In [ ]:
print("Top terms per cluster:")

order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    print("Cluster %d words:" % i +'')
    for ind in order_centroids[i, :6]:
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print()
    print()
    print("Cluster %d titles:" % i, end='')
    for title in frame.ix[i]['title'].values.tolist():
        print(' %s,' % title, end='')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

dist = 1 - cosine_similarity(tfidf_matrix)

In [150]:
new_df[new_df.clusters==1][['title','genre','key_words']]

,title,genre,key_words
1,1492 conquest of paradise,"['adventure', 'biography', 'drama', 'history']",me you them us it man colon something ocean go...
1,the fifth element,"['action', 'adventure', 'romance', 'sci-fi', '...",it me you stones father life leeloo president ...
1,air force one,"['action', 'drama', 'thriller']",it you us me president air force mr family ter...
1,airplane ii the sequel,"['comedy', 'romance', 'sci-fi']",it simon you me people us word him commissione...
1,airplane,"['comedy', 'romance']",it you us him me something anything ship thing...
1,amadeus,"['biography', 'drama', 'music']",you it me him herr mozart what us people money...
1,american madness,['drama'],you it dickson me mr him us anything money som...
1,barry lyndon,"['drama', 'romance', 'war']",you me it him us something people everything a...
1,dark star,"['comedy', 'sci-fi', 'thriller']",it me you something anything # i bomb everythi...
1,galaxy quest,"['action', 'adventure', 'comedy', 'sci-fi']",it i you me us something him them nothing comm...


In [370]:
from gensim.models import word2vec

model = word2vec.Word2Vec(
    sentences,
    workers=4,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=10,  # Minimum word count threshold.
    window=6,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

0      did change hair? no. might wanna think missed ...
1      is man knew, treasurer sanchez? yes, majesty. ...
2      just do. say thing say. open mouth. okay. fool...
3      how do, mr. miller? terribly sorry. way meet y...
4      maybe shoulda stole better truck, tonto. got r...
5      lord forgive me.. already know too, much.. ".....
6      welcome home. know much missed you? smoking ? ...
7      something wrong stairs? avoid-all-contact-day....
8      hey, wrong let see smile. kind long night. dan...
9      i sorry screw-up. help it late go trick-or-tre...
10     we already inspected. sir, plane carries presi...
11     commissioner, know mercury shuttle needs anoth...
12     jack, fred bliffert blue turtleneck? maybe fli...
13     hope plumbing same. is. go? put wash-and-wear ...
14     - today? terrible. - terrible? better yesterda...
15     what want? father vogler. chaplain here. thoug...
16     are cold? yes. good. jack. david. good time yo...
17     you carrying much money 

In [384]:
data = df_clean['dialogue'].apply(lambda x: x.split())

In [385]:

id2word = corpora.Dictionary(data)

# Create Corpus
texts = data

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 7), (16, 11), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 5), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 3), (36, 1), (37, 2), (38, 2), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 2), (47, 1), (48, 1), (49, 1), (50, 1), (51, 2), (52, 1), (53, 2), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 9), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 3), (70, 5), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 3), (82, 1), (83, 1), (84, 2), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 2), (96, 2), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 3), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 2), (109, 1), (110, 1

In [386]:
id2word[0]

'"'

In [387]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('"', 1),
  ('"angry,', 1),
  ('"any', 1),
  ('"cool"?', 1),
  ('"everyone', 1),
  ('"gloria', 1),
  ('"heinous', 1),
  ('"hot', 1),
  ('"i', 1),
  ('"if', 1),
  ('"laid', 1),
  ('"non-smoker"?', 1),
  ('"occupied".', 1),
  ('"persona"?', 1),
  ('"real', 1),
  ('-', 7),
  ('.', 11),
  ('..the', 1),
  ('0.', 1),
  ('7-eleven', 1),
  ('90210."', 1),
  ('9th.', 1),
  ('?', 5),
  ('a-list.', 1),
  ('a.', 1),
  ('able', 1),
  ('about?', 2),
  ('absence?', 1),
  ('absolutely', 1),
  ('abused,', 1),
  ('act', 2),
  ('acting', 1),
  ('action.', 1),
  ('activity.', 1),
  ('acts', 1),
  ('actually', 3),
  ('actually,', 1),
  ('actually.', 2),
  ('ad', 2),
  ('admiral?', 1),
  ('admit', 1),
  ('adorable!', 1),
  ('advice', 1),
  ('affection', 1),
  ('afraid', 1),
  ('afterwards,', 1),
  ('again.', 2),
  ('again?', 1),
  ('agent', 1),
  ('agree', 1),
  ('alcatraz', 1),
  ('all,', 2),
  ('all.', 1),
  ('allowed', 2),
  ('allows.', 1),
  ('allure', 1),
  ('alone', 1),
  ('alone.', 1),
  ('already'

In [380]:
import gensim

In [388]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [389]:
# Print the Keyword in the 10 topics
for topic, weights in lda_model.print_topics():
    print(topic)
    print(weights)
doc_lda = lda_model[corpus]


0
0.008*"mister" + 0.005*"buddy" + 0.003*"n" + 0.003*"goddam" + 0.003*"ya" + 0.003*"charley" + 0.002*"mcmurphy." + 0.002*"charley." + 0.002*"edie," + 0.002*"johnny"
1
0.009*"know" + 0.007*"get" + 0.006*"like" + 0.006*"oh," + 0.006*"well," + 0.005*"think" + 0.005*"got" + 0.005*"it." + 0.005*"no," + 0.004*"you."
2
0.013*"mary" + 0.011*"simone" + 0.005*"clarence," + 0.004*"viktor." + 0.004*"viktor" + 0.004*"clarence." + 0.004*"/" + 0.004*"simone." + 0.004*"ninotchka," + 0.004*"reverend"
3
0.010*"know" + 0.010*"like" + 0.009*"get" + 0.008*"got" + 0.007*"want" + 0.006*"think" + 0.006*"it." + 0.006*"you." + 0.006*"one" + 0.006*"going"
4
0.008*"agent" + 0.008*"low" + 0.006*"suspect" + 0.006*"koessler" + 0.005*"richard" + 0.005*"j.j.," + 0.004*"j.j." + 0.004*"susie" + 0.004*"susie," + 0.004*"chat"
5
0.010*"dr." + 0.006*"rose," + 0.005*"would" + 0.004*"yu" + 0.003*"rose" + 0.003*"rose." + 0.003*"li" + 0.003*"-" + 0.003*"jack." + 0.003*"you."
6
0.045*"." + 0.009*"know" + 0.009*"get" + 0.007*"got

In [174]:

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1] if feature_names[i] not in my_stopwords])
        print(message)
    print()

n_samples = 2000
n_features = 500
n_components = 10
n_top_words = 50


# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.

t0 = time()
data_samples = comedies
print("done in %0.3fs." % (time() - t0))

# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d..."
      % (n_samples))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LDA(n_components=n_components, max_iter=5,
                                learning_method='batch',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

done in 0.000s.
Extracting tf-idf features for NMF...
done in 1.405s.
Extracting tf features for LDA...
done in 1.257s.

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=2000...
done in 0.105s.

Topics in NMF model (Frobenius norm):
Topic #0: okay gonna night guy things god great day believe years big told money old feel kind away place hey girl bad guess hell wait dead home thank mother fine course remember real getting ask stop boy wrong listen world tonight talking job mind thanks kill understand house mr leave
Topic #1: dr ok doctor darling thank god hello miss night sort world dad hi story office frank woman apartment happened wife piece fine sir quite dog fucking fun everybody evening goddamn excuse funny fuck fuckin general game gave exactly george gets friends girl girls giving glad getting face friend feet fact
Topic #2: ed al eddie gonna money general gave game funny fucking fun gets fuckin fuck friends friend george girls getting girl frank giving glad 

In [172]:
lda.transform(tfidf)

array([[0.00637955, 0.00637967, 0.00637975, ..., 0.00637964, 0.00637978,
        0.00637977],
       [0.00694002, 0.00694023, 0.00694076, ..., 0.00693943, 0.00694019,
        0.00694078],
       [0.01690532, 0.01690437, 0.01690193, ..., 0.01689758, 0.8478793 ,
        0.01691422],
       ...,
       [0.00583681, 0.00583677, 0.39578833, ..., 0.00583651, 0.00583702,
        0.005837  ],
       [0.93522737, 0.00719696, 0.00719652, ..., 0.00719702, 0.00719717,
        0.00719678],
       [0.00810178, 0.00810178, 0.00810023, ..., 0.00810014, 0.92709361,
        0.00810097]])

In [935]:
keyphrases= []
for noun in df_clean['docs'][0].noun_chunks:
    if len(noun.text.split())>1:
        keyphrases.append(noun.text)

In [943]:
important_sentences =[]
for sent in df_clean['docs'][0].sents:
    sentence = str(sent)
    for phrase in keyphrases:
        if sentence.find(phrase)!= -1:
            important_sentences.append(sentence)
            break

In [954]:
for i,sent in enumerate(important_sentences):
    important_sentences[i] = re.sub(r"[\?\.\!\-]", " ", sent)

In [956]:
ps = PorterStemmer()
fragments = [sent.split() for sent in important_sentences]
for words in fragments:
    for word in words:
        print(ps.stem(word))

say
expos
group
freshmen
girl
teeth
zipper
new
guy
tell
c
mon
mani
peopl
old
school
mani
peopl
go
here
jack
clint
eastwood
allow
date
older
sister
doe
katarina
stratford
express
opinion
terrorist
action
way,
bobbi
rictor
gonad
retriev
oper
went
quit
well,
case
interest
still
maintain
kick
ball
mere
spectat
believ
"heinou
bitch"
term
use
often
patrick
verona
random
skid
pat
verona
heard
porn
movi
sure
complet
incap
anyth
interest
alway
look
block
e
mandella,
eat
starv
slow
way
die
realiz
men
fine
institut
sever
lacking,
kill
william
shakespear
beyond
scope
normal
teenag
obsess
ventur
far
past
daytim
talk
show
fodder
enter
world
need
expens
therapi
gene
pool
rare
dilut
alway
shit
eat
grin
joey
dorsey
wish
could
say
moron,
number
twelv
class
mostli
region
stuff,
rumor
big
tube
sock
ad
come
out
vintag
look
over,
kat
notic
part
featur
big
kmart
spread
elbow
tough
break
sure
william
friend
minor
encount
shrew
bianca
sister
mewling,
rampalian
wretch
herself
nowher
hi,
daddi
sarah
lawrenc
side

In [40]:
for movie_id, x in tqdm(zip(movie_ids, range(df.shape[0]))):
        sentences[movie_id] = [sent for sent in df['processed_dialogue'][x].sents]

617it [00:01, 357.71it/s]


In [62]:
sentences['m616'][0]

zulu dawn  BLOOMFIELD  Zulu may not wear shoes or trousers and the like but it don '

In [992]:
type(df_clean['docs'][0].noun_chunks)

generator

In [1007]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [noun.text for noun in text.noun_chunks ]
   
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(500)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

In [1215]:
# Set up the bags.
common_words = []
for x in tqdm(range(df_clean.shape[0])):
    dialogue = bag_of_words(df_clean['docs'][x])
# Combine bags to create a set of unique words.
    common_words += dialogue
    
most_common_words = set(common_words)







  0%|          | 0/592 [00:00<?, ?it/s]





  1%|▏         | 8/592 [00:00<00:35, 16.35it/s]





  2%|▏         | 14/592 [00:00<00:28, 20.08it/s]





  3%|▎         | 19/592 [00:00<00:24, 22.99it/s]





  4%|▎         | 22/592 [00:00<00:26, 21.77it/s]





  5%|▍         | 27/592 [00:01<00:21, 25.81it/s]





  5%|▌         | 30/592 [00:01<00:24, 22.68it/s]





  6%|▌         | 34/592 [00:01<00:23, 24.13it/s]





  6%|▋         | 37/592 [00:01<00:22, 24.47it/s]





  7%|▋         | 40/592 [00:01<00:22, 24.37it/s]





  7%|▋         | 43/592 [00:01<00:28, 19.41it/s]





  8%|▊         | 46/592 [00:01<00:28, 18.88it/s]





  8%|▊         | 49/592 [00:02<00:28, 19.17it/s]





  9%|▉         | 52/592 [00:02<00:28, 19.14it/s]





  9%|▉         | 56/592 [00:02<00:26, 20.51it/s]





 10%|▉         | 59/592 [00:03<00:49, 10.76it/s]





 10%|█         | 61/592 [00:03<00:43, 12.32it/s]





 11%|█         | 64/592 [00:03<00:42, 12.30it/s]





 11%|█         | 66/592 [00:03<0

In [1216]:
most_common_stems =[]
ps =PorterStemmer()
for phrase in tqdm(most_common_words):
    word = phrase.split()[-1]
    if word not in my_stopwords:
        most_common_stems.append(ps.stem(word))







  0%|          | 0/122142 [00:00<?, ?it/s]





  2%|▏         | 2462/122142 [00:00<00:04, 24616.80it/s]





  4%|▍         | 4805/122142 [00:00<00:04, 24245.91it/s]





  6%|▌         | 7073/122142 [00:00<00:04, 23752.85it/s]





  8%|▊         | 9204/122142 [00:00<00:04, 22961.73it/s]





  9%|▉         | 11408/122142 [00:00<00:04, 22675.96it/s]





 11%|█         | 13714/122142 [00:00<00:04, 22787.46it/s]





 13%|█▎        | 16075/122142 [00:00<00:04, 22988.85it/s]





 15%|█▌        | 18481/122142 [00:00<00:04, 23299.51it/s]





 17%|█▋        | 20737/122142 [00:00<00:04, 23071.91it/s]





 19%|█▉        | 23022/122142 [00:01<00:04, 23004.53it/s]





 21%|██        | 25256/122142 [00:01<00:04, 22750.84it/s]





 23%|██▎       | 27508/122142 [00:01<00:04, 22680.12it/s]





 24%|██▍       | 29744/122142 [00:01<00:04, 21169.55it/s]





 26%|██▌       | 31901/122142 [00:01<00:04, 21284.77it/s]





 28%|██▊       | 34028/122142 [00:01<00:04, 20475.86it/s]





 30%|

In [1217]:
most_common_stems=list(set(most_common_stems))

In [1218]:
len(most_common_stems)

19730

In [1219]:
common_features = list(sorted(most_common_stems))

In [1227]:
for feat in common_features:
    for char in feat:
        if char.isalpha() ==False:
            common_features.remove(feat)
            break

In [1228]:
len(common_features)

16998

In [1229]:
for word in tqdm(common_features[:5000]):
    df_clean[word] = 0
  







  0%|          | 0/5000 [00:00<?, ?it/s]





  4%|▎         | 184/5000 [00:00<00:02, 1834.94it/s]





  8%|▊         | 376/5000 [00:00<00:02, 1857.36it/s]





 10%|█         | 519/5000 [00:00<00:02, 1702.41it/s]





 13%|█▎        | 641/5000 [00:00<00:02, 1520.67it/s]





 15%|█▌        | 769/5000 [00:00<00:02, 1439.33it/s]





 18%|█▊        | 887/5000 [00:00<00:03, 1199.67it/s]





 20%|█▉        | 996/5000 [00:00<00:03, 1049.89it/s]





 22%|██▏       | 1097/5000 [00:00<00:04, 896.96it/s]





 24%|██▍       | 1196/5000 [00:01<00:04, 921.66it/s]





 26%|██▌       | 1309/5000 [00:01<00:03, 975.16it/s]





 28%|██▊       | 1410/5000 [00:01<00:03, 984.64it/s]





 30%|███       | 1515/5000 [00:01<00:03, 1002.25it/s]





 32%|███▏      | 1616/5000 [00:01<00:03, 933.90it/s] 





 34%|███▍      | 1711/5000 [00:01<00:03, 890.51it/s]





 36%|███▌      | 1802/5000 [00:01<00:03, 835.48it/s]





 38%|███▊      | 1888/5000 [00:01<00:03, 808.59it/s]





 40%|███▉      | 1

In [1231]:
  
for i, sentence in tqdm(enumerate(df_clean['docs'])):
        
        # I already converted the sentence to lemmas, and filtered out punctuation and stop words.
        # Now I'm just going to get a list of the words in each line that are in the common words list.
        words = [token.lemma_
                 for token in sentence
                 if token.lemma_ in common_features[:5000]
                ]
        
        # Populate the row with word counts
        for word in words:
            df_clean.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))







0it [00:00, ?it/s]





1it [00:00,  1.09it/s]

Processing row 0








2it [00:01,  1.29it/s]





3it [00:02,  1.04it/s]





4it [00:04,  1.08s/it]





5it [00:04,  1.00it/s]





6it [00:06,  1.20s/it]





7it [00:06,  1.09it/s]





8it [00:44, 12.02s/it]





9it [00:48,  9.47s/it]





10it [00:48,  6.82s/it]





11it [00:49,  4.94s/it]





12it [00:51,  3.97s/it]





13it [00:52,  3.05s/it]





14it [00:58,  3.97s/it]





15it [01:00,  3.56s/it]





16it [01:03,  3.29s/it]





17it [01:04,  2.70s/it]





18it [01:06,  2.31s/it]





19it [01:12,  3.55s/it]





20it [01:25,  6.24s/it]





21it [01:26,  4.90s/it]





22it [01:28,  3.97s/it]





23it [01:29,  3.05s/it]





24it [01:31,  2.78s/it]





25it [01:32,  2.22s/it]





26it [01:34,  2.23s/it]





27it [01:37,  2.39s/it]





28it [01:50,  5.62s/it]





29it [01:53,  4.70s/it]





30it [01:54,  3.53s/it]





31it [01:55,  2.93s/it]





32it [01:56,  2.27s/it]





33it [02:00,  2.69s/it]





34it [02:06,  3.79s/it]





35it [02:13,  4.85s/it]





36it [02:15,  3.

Processing row 50








52it [03:39,  5.46s/it]





53it [03:44,  5.22s/it]





54it [03:47,  4.57s/it]





55it [03:48,  3.56s/it]





56it [03:57,  5.15s/it]





57it [04:02,  4.93s/it]





58it [04:03,  3.81s/it]





59it [04:04,  2.98s/it]





60it [04:13,  4.73s/it]





61it [04:14,  3.77s/it]





62it [04:17,  3.37s/it]





63it [04:18,  2.87s/it]





64it [04:34,  6.76s/it]





65it [04:42,  7.11s/it]





66it [04:46,  6.03s/it]





67it [04:47,  4.64s/it]





68it [04:50,  4.20s/it]





69it [04:52,  3.33s/it]





70it [04:54,  2.93s/it]





71it [04:54,  2.28s/it]





72it [04:56,  2.02s/it]





73it [04:58,  2.08s/it]





74it [05:11,  5.36s/it]





75it [05:13,  4.35s/it]





76it [05:17,  4.23s/it]





77it [05:19,  3.71s/it]





78it [05:25,  4.21s/it]





79it [05:31,  4.66s/it]





80it [05:33,  4.14s/it]





81it [05:35,  3.30s/it]





82it [05:38,  3.29s/it]





83it [05:38,  2.37s/it]





84it [05:39,  1.86s/it]





85it [05:43,  2.58s/it]





86it [05

...







452it [25:31,  2.64s/it]





453it [25:32,  2.19s/it]





454it [25:39,  3.57s/it]





455it [25:40,  2.74s/it]





456it [25:44,  3.15s/it]





457it [25:45,  2.53s/it]





458it [25:46,  2.09s/it]





459it [25:46,  1.57s/it]





460it [25:48,  1.55s/it]





461it [25:49,  1.46s/it]





462it [25:50,  1.37s/it]





463it [25:53,  1.92s/it]





464it [25:55,  1.83s/it]





466it [25:59,  1.85s/it]





467it [26:04,  2.94s/it]





468it [26:09,  3.49s/it]





469it [26:10,  2.62s/it]





470it [26:11,  2.31s/it]





471it [26:13,  2.11s/it]





472it [26:14,  1.77s/it]





473it [26:15,  1.64s/it]





474it [26:18,  1.88s/it]





475it [26:20,  1.97s/it]





476it [26:22,  2.08s/it]





477it [26:23,  1.80s/it]





478it [26:25,  1.71s/it]





479it [26:25,  1.39s/it]





480it [26:27,  1.30s/it]





481it [26:28,  1.34s/it]





482it [26:31,  1.79s/it]





483it [26:33,  1.85s/it]





484it [26:34,  1.61s/it]





485it [26:38,  2.29s/it]





486i

Processing row 500








502it [27:19,  2.21s/it]





503it [27:21,  2.04s/it]





504it [27:22,  1.75s/it]





505it [27:24,  1.85s/it]





506it [27:42,  6.66s/it]





507it [27:44,  5.28s/it]





508it [27:45,  4.03s/it]





509it [27:47,  3.42s/it]





510it [27:50,  3.10s/it]





511it [27:52,  2.77s/it]





512it [27:54,  2.64s/it]





513it [27:54,  1.96s/it]





514it [27:56,  1.95s/it]





515it [27:59,  2.22s/it]





516it [28:00,  1.81s/it]





517it [28:02,  1.79s/it]





518it [28:03,  1.56s/it]





519it [28:04,  1.50s/it]





520it [28:05,  1.31s/it]





521it [28:06,  1.16s/it]





522it [28:09,  1.81s/it]





523it [28:10,  1.64s/it]





524it [28:14,  2.12s/it]





525it [28:14,  1.68s/it]





526it [28:19,  2.56s/it]





527it [28:19,  1.95s/it]





528it [28:21,  1.73s/it]





529it [28:21,  1.45s/it]





530it [28:23,  1.51s/it]





531it [28:25,  1.50s/it]





532it [28:26,  1.52s/it]





533it [28:27,  1.32s/it]





534it [28:28,  1.28s/it]





535i

Processing row 550








552it [29:05,  1.39s/it]





553it [29:07,  1.45s/it]





554it [29:09,  1.56s/it]





555it [29:11,  1.81s/it]





556it [29:13,  1.90s/it]





557it [29:14,  1.60s/it]





558it [29:14,  1.18s/it]





559it [29:16,  1.25s/it]





560it [29:17,  1.16s/it]





561it [29:18,  1.35s/it]





562it [29:20,  1.51s/it]





563it [29:21,  1.41s/it]





564it [29:25,  1.96s/it]





565it [29:28,  2.52s/it]





566it [29:32,  2.72s/it]





567it [29:35,  2.82s/it]





568it [29:36,  2.26s/it]





569it [29:39,  2.45s/it]





570it [29:42,  2.73s/it]





571it [29:44,  2.60s/it]





572it [29:46,  2.42s/it]





573it [29:48,  2.30s/it]





574it [29:49,  1.83s/it]





575it [29:50,  1.60s/it]





576it [29:53,  1.89s/it]





577it [29:55,  2.10s/it]





578it [29:56,  1.77s/it]





579it [29:57,  1.58s/it]





581it [29:59,  1.29s/it]





582it [30:00,  1.38s/it]





583it [30:05,  2.30s/it]





584it [30:06,  2.03s/it]





585it [30:07,  1.67s/it]





586i

In [1039]:
df_comedy['genre'] = df_comedy['genre'].astype(str)

In [1289]:
df_comedy.head()
y = np.where(df_comedy['genre'].str.contains('comedy'), 1, 0)

In [1256]:
df_clean =df_clean.merge(df[['ids','clean_year','clean_rating']], on='ids')

In [1290]:
df_comedy =df_comedy.merge(df[['ids','clean_year','clean_rating']], on='ids')

In [1323]:
#X =df_clean.iloc[:, 11:5011]
X = df_comedy.iloc[:, 7:8734]

In [1327]:
svd= TruncatedSVD(500)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data, then project the training data.
X_svd = lsa.fit_transform(X)

variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

#Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
paras_by_component=pd.DataFrame(X_svd)

Percent variance captured by all components: 100.00000000000017


In [1328]:
#X = df_clean[['clean_year', 'clean_rating']].join(paras_by_component)
#X =df_comedy[['clean_year', 'clean_rating']].join(paras_by_component)

In [1329]:
y = np.where(df_comedy['genre'].str.contains('comedy'), 1, 0)

In [1330]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=0) 

In [1331]:
y_test.mean()

0.3516483516483517

In [1332]:
lr = LogisticRegression()

In [1333]:
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [1334]:
lr.score(X_test, y_test)

0.6923076923076923

In [1335]:
cross_val_score(lr, X_test, y_test, cv=10)

array([0.7       , 0.7       , 0.66666667, 0.33333333, 0.66666667,
       0.44444444, 0.44444444, 0.66666667, 0.77777778, 0.875     ])

In [1336]:
rfc = RandomForestClassifier()

In [1337]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [1338]:
rfc.score(X_test, y_test)
#cross_val_score(rfc, X_test, y_test, cv=10)

0.6593406593406593

In [1339]:
gb = GradientBoostingClassifier(max_depth=2, learning_rate=0.05, min_impurity_decrease=0.05)

In [1340]:
gb.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=2,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.05, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [1341]:
gb.score(X_test, y_test)

0.7692307692307693

In [1342]:
cross_val_score(gb, X_test, y_test, cv=5)

array([0.47368421, 0.63157895, 0.77777778, 0.44444444, 0.64705882])

In [1348]:
from xgboost import XGBClassifier

In [1349]:
xgb = XGBClassifier()

xgb.fit(X_train, y_train)

In [1321]:
xgb.score(X_test, y_test)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.6373626373626373